In [1]:
import pandas as pd

In [2]:
defenses = pd.read_csv("weeks_frame1/frame1_process/defense_players_plays.csv")
defenses.head()

,Unnamed: 0,x,y,s,a,dis,o,dir,nflId,displayName,jerseyNumber,position,team,playDirection,id
0,1,88.89,36.47,0.01,0.01,0.01,105.63,66.66,79848.0,Malcolm Jenkins,27.0,SS,home,left,2018090600:75:1
1,3,86.31,22.01,0.09,0.42,0.01,70.12,168.91,2495613.0,Corey Graham,24.0,FS,home,left,2018090600:75:1
2,5,73.64,28.70,0.03,0.02,0.01,103.05,219.41,2534832.0,Rodney McLeod,23.0,FS,home,left,2018090600:75:1
3,7,86.48,31.12,0.09,0.03,0.04,95.90,33.36,2552315.0,Jordan Hicks,58.0,MLB,home,left,2018090600:75:1
4,8,82.67,20.53,0.03,0.07,0.01,81.14,174.57,2552689.0,Ronald Darby,21.0,CB,home,left,2018090600:75:1


In [3]:
plays = pd.read_csv("raw_data/plays.csv")
plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,...,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,...,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,...,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,...,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False


In [4]:
games = pd.read_csv("raw_data/games.csv")
games.head()

,gameId,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week
0,2018090600,09/06/2018,20:20:00,PHI,ATL,1
1,2018090901,09/09/2018,13:00:00,CLE,PIT,1
2,2018090902,09/09/2018,13:00:00,IND,CIN,1
3,2018090903,09/09/2018,13:00:00,MIA,TEN,1
4,2018090900,09/09/2018,13:00:00,BAL,BUF,1


In plays, absoluteYardline is the most important column for now.

![title](resources/field.png)

First we are going to identify how many players are there in the deepzone. We are going to divide the defenses zones like the below picture.

![title](resources/zonas_defensivas.jpg)

The deep zone starts 15 yards from the line of scrimmage. It would be interesting to start with no red zones defenses, and then expand it. So let's identify no red zone defenses.

The red zone is when the attack team is inside the 20 yard of the opponent team.

In [5]:
plays_processed = plays.copy()
plays_processed.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,...,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,...,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,...,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,...,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False


We have to remove those plays we remove in the other dataset.
{2: 329,
 3: 2100,
 4: 544,
 5: 1558,
 8: 1243,
 9: 1544,
 10: 2472,
 11: 1165,
 12: 2125,
 13: 1137,
 14: 1570,
 15: 3142,
 17: 2739}

In [7]:
plays_processed.drop(plays_processed[(plays_processed["playId"]==2100) & (plays_processed["gameId"]==2018092309)].index,inplace=True)
plays_processed.drop(plays_processed[(plays_processed["playId"]==1243) & (plays_processed["gameId"]==2018102810)].index,inplace=True)
plays_processed.drop(plays_processed[(plays_processed["playId"]==2100) & (plays_processed["gameId"]==2018120200)].index,inplace=True)

In [35]:
plays_processed.drop(plays_processed[(plays_processed["playId"]==329) & (plays_processed["gameId"]==2018091607)].index,inplace=True)
plays_processed.drop(plays_processed[(plays_processed["playId"]==544) & (plays_processed["gameId"]==2018093011)].index,inplace=True)
plays_processed.drop(plays_processed[(plays_processed["playId"]==1558) & (plays_processed["gameId"]==2018100700)].index,inplace=True)
plays_processed.drop(plays_processed[(plays_processed["playId"]==1544) & (plays_processed["gameId"]==2018110402)].index,inplace=True)
plays_processed.drop(plays_processed[(plays_processed["playId"]==2472) & (plays_processed["gameId"]==2018111104)].index,inplace=True)
plays_processed.drop(plays_processed[(plays_processed["playId"]==2125) & (plays_processed["gameId"]==2018112509)].index,inplace=True)
plays_processed.drop(plays_processed[(plays_processed["playId"]==1570) & (plays_processed["gameId"]==2018120901)].index,inplace=True)
plays_processed.drop(plays_processed[(plays_processed["playId"]==3142) & (plays_processed["gameId"]==2018121611)].index,inplace=True)
plays_processed.drop(plays_processed[(plays_processed["playId"]==2739) & (plays_processed["gameId"]==2018123012)].index,inplace=True)

In [46]:
plays_processed.drop(plays_processed[(plays_processed["playId"]==1539) & (plays_processed["gameId"]==2018092000)].index,inplace=True)

In [47]:
for index,row in plays_processed.iterrows():
    week = games[games["gameId"]==row["gameId"]]["week"].values[0]
    playId = str(row["gameId"])+":"+str(row["playId"])+":"+str(week)
    print(playId)
    direction_play = defenses[defenses["id"]==str(playId)]["playDirection"].values[0]
    if row["absoluteYardlineNumber"] > 10 and row["absoluteYardlineNumber"] <= 30 and direction_play == "left":
        print("RZ!! on {}".format(playId))

2018090600:75:1
2018090600:146:1
2018090600:168:1
2018090600:190:1
2018090600:256:1
RZ!! on 2018090600:256:1
2018090600:320:1
2018090600:344:1
2018090600:402:1
2018090600:492:1
2018090600:521:1
RZ!! on 2018090600:521:1
2018090600:545:1
RZ!! on 2018090600:545:1
2018090600:636:1
RZ!! on 2018090600:636:1
2018090600:752:1
2018090600:776:1
2018090600:839:1
2018090600:889:1
2018090600:949:1
2018090600:992:1
2018090600:1037:1
2018090600:1061:1
2018090600:1085:1
2018090600:1202:1
RZ!! on 2018090600:1202:1
2018090600:1295:1
RZ!! on 2018090600:1295:1
2018090600:1344:1
RZ!! on 2018090600:1344:1
2018090600:1423:1
2018090600:1473:1
2018090600:1546:1
2018090600:1568:1
2018090600:1628:1
2018090600:1680:1
2018090600:1750:1
2018090600:1779:1
2018090600:1801:1
2018090600:1869:1
2018090600:1894:1
2018090600:1916:1
2018090600:1940:1
2018090600:2055:1
2018090600:2094:1
2018090600:2116:1
2018090600:2260:1
2018090600:2360:1
2018090600:2382:1
2018090600:2450:1
2018090600:2474:1
2018090600:2503:1
2018090600:25

IndexError: index 0 is out of bounds for axis 0 with size 0

In [38]:
plays[plays["playId"]==1539]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
1420,2018091600,1539,(2:30) (Shotgun) C.Newton pass incomplete deep...,2,1,10,CAR,play_type_pass,CAR,36,...,10.0,02:30:00,46.0,NaN,NaN,I,0,0,-0.302325,False
2261,2018092000,1539,(5:30) (Shotgun) T.Taylor pass short right to ...,2,3,12,CLE,play_type_pass,CLE,39,...,0.0,05:30:00,71.0,NaN,NaN,C,12,12,1.953195,False
4047,2018093007,1539,(2:22) (Shotgun) C.Wentz pass short left to Z....,2,1,10,PHI,play_type_pass,PHI,14,...,3.0,02:22:00,24.0,NaN,NaN,C,2,2,-0.136774,False
6013,2018101402,1539,(6:43) P.Rivers pass deep left to Ty.Williams ...,2,1,10,LAC,play_type_pass,CLE,45,...,3.0,06:43:00,65.0,NaN,NaN,C,45,45,3.886953,False
6340,2018101408,1539,(3:14) (Shotgun) C.Newton pass incomplete shor...,2,1,10,CAR,play_type_pass,WAS,23,...,17.0,03:14:00,87.0,NaN,NaN,I,0,0,-0.201356,False
14248,2018120901,1539,(8:42) (Shotgun) J.Goff pass short right to T....,2,3,15,LA,play_type_pass,LA,20,...,6.0,08:42:00,30.0,NaN,NaN,C,8,8,-0.916715,False
16712,2018122307,1539,(5:30) (Shotgun) M.Stafford pass short right t...,2,1,10,DET,play_type_pass,MIN,35,...,6.0,05:30:00,75.0,NaN,NaN,C,7,7,0.127970,False


In [51]:
defenses[defenses["id"]=="2018090600:146:1"]["playDirection"].values[0]

'left'

In [48]:
week3 = pd.read_csv("weeks_frame1/frame1_raw/week3_frame1.csv")
week3[week3["playId"]==1553]

,Unnamed: 0,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
5192,329688,2018-09-23T18:07:53.900Z,46.05,16.56,0.00,0.00,0.00,NaN,NaN,line_set,2540164.0,Tony Jefferson,23.0,SS,1,home,2018092301,1553,right,NaN
5193,329689,2018-09-23T18:07:59.400Z,45.53,15.10,0.84,0.50,0.09,280.96,68.39,None,4365.0,Brandon Carr,24.0,CB,1,home,2018092301,1553,right,NaN
5194,329690,2018-09-23T18:07:59.400Z,35.80,15.14,2.67,2.62,0.28,90.17,178.02,None,497322.0,Emmanuel Sanders,10.0,WR,1,away,2018092301,1553,right,NaN
5195,329691,2018-09-23T18:07:59.400Z,37.59,16.47,0.00,0.00,0.00,96.08,136.08,None,497328.0,Demaryius Thomas,88.0,WR,1,away,2018092301,1553,right,NaN
5196,329692,2018-09-23T18:07:59.400Z,55.37,27.55,0.62,0.60,0.07,239.98,198.64,None,2495775.0,Eric Weddle,32.0,FS,1,home,2018092301,1553,right,NaN
5197,329693,2018-09-23T18:07:59.400Z,39.66,29.92,0.07,0.04,0.00,259.03,214.68,None,2505660.0,Terrell Suggs,55.0,OLB,1,home,2018092301,1553,right,NaN
5198,329694,2018-09-23T18:07:59.400Z,39.17,20.91,0.28,0.28,0.04,331.94,215.16,None,2508004.0,Anthony Levine,41.0,DB,1,home,2018092301,1553,right,NaN
5199,329695,2018-09-23T18:07:59.400Z,33.04,23.92,0.01,0.01,0.01,96.44,97.87,None,2532888.0,Case Keenum,4.0,QB,1,away,2018092301,1553,right,NaN
5200,329696,2018-09-23T18:07:59.400Z,39.39,19.72,0.00,0.00,0.00,84.83,142.96,None,2552279.0,Za'Darius Smith,90.0,OLB,1,home,2018092301,1553,right,NaN
5201,329697,2018-09-23T18:07:59.400Z,35.31,19.36,0.00,0.00,0.00,145.37,232.53,None,2552399.0,Jeff Heuerman,82.0,TE,1,away,2018092301,1553,right,NaN
